In [1]:
"""
https://irbank.net/download
からcsvデータを取得して、決算情報等を更新して学習するときに便利なようにするprogram
init   : 2021/10/26 init
update : 2021/11/03 select_dd & timeSeries making ---> 
update : 2021/12/03 select_dd2 (前後数週間の増減について)

"""
import os
# os.makedirs(new_dir_path_recursive, exist_ok=True)
import sys
from datetime import datetime,timedelta
import pandas as pd
import  numpy as np
import subprocess
from tqdm import tqdm
#science  ----------------
# from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
# plt.rcParams['font.family'] = 'Meiryo'

# from matplotlib.animation import FuncAnimation
import numpy as np
# from scipy.integrate._ivp.radau import P
# from scipy.integrate._ivp.radau import P
from sklearn.preprocessing import MinMaxScaler
import time
import warnings 
warnings.simplefilter("ignore")
import json
import pandas as pd
import plotly
import plotly.express as px

# stock module ----------------
HOME="/home/ysorimachi/work/sori_py2/COVID-19"
SOME="/home/ysorimachi/work/sori_py2/analysis_stock"
sys.path.append(f"{SOME}/py_covid")
from utils import *

# from utils_data import check_cate, get_stock

In [2]:
freq = 7
n=10
df_c, df_d, df_r = select_dd2(dd="20211130",freq=freq,n=n) #2021.12.02

In [3]:
# df_c = df_c.sort_values("Confirmed-0(~1130)",ascending=False)
# df_c.head(100)

In [7]:
def mk_diff(df,col ="Confirmed"):
    #　新規感染者数の平均
    use_col = [ c for c in df.columns if col in c]
    df = df[use_col].T
    df = df.iloc[::-1,:]

    df = df.diff().T / freq
    return df

def plotly_map(df,col,vmin=None,vmax=None,color='RdBu_r'):
    """
     < 2021.12.03 > 
    location name = [ iso3 format ]
    color = 'Jet'
    color = 'RdBu_r'
    """
    if vmin==None:
        vmin = np.min(df[col])
        vmax = np.max(df[col])
        
    fig = px.choropleth(
        df,locations = "iso3",
        color=col,#Deaths
#         hover_name=[col,"rank"],
        hover_name="rank",
        range_color=[vmin,vmax],
                                # animation_frame='date',
                              projection="natural earth",
                              color_continuous_scale=color
                              # https://plotly.com/python/builtin-colorscales/
        )
      # sys.exit()
    fig.show()
    return

In [10]:

def plot_world_ranking(col="Confirmed",ylim=None,N=None):
    if col =="Confirmed":
        df = mk_diff(df_c,col =col)
    if col =="Deaths":
        df = mk_diff(df_d,col =col)
        
    df = df.sort_values(f"{col}-0(~1130)",ascending=False)
    df["rank"] = np.arange(len(df)) + 1
    df.index.name = "iso3"
    df = df.reset_index()
    if N:
        df = df.iloc[:N,:]
    print("shape=>",df.shape)
    if ylim:
        vmin,vmax = ylim
    else:
        vmin,vmax = None,None
    plotly_map(df,col=f"{col}-0(~1130)",vmin=vmin,vmax=vmax,color='Jet')
    return 
#---------

col = "Confirmed"
# col = "Deaths"
plot_world_ranking(col="Confirmed",ylim=[None,None],N=100)

shape=> (100, 12)
